In [4]:
import os
import sys
import nltk
import itertools 

In [5]:
# stupid way to access the data
my_path = '/Users/vassiki/Desktop/nhw17/DataDrivenCognitiveOntology/'
sys.path.append(my_path)

In [6]:
# Import core functions
from lisc.base import Base
from lisc.core.db import SCDB
from lisc.data import Data
from lisc.data_all import DataAll

In [7]:
# Add data to path
db = SCDB(False)
db.project_path = os.path.join(os.getcwd(), 'dat')
db.gen_paths()

In [8]:
# Using a single case, for autobiographical recall
# Need associated .json file to run this!
dat = Data('autobiographical recall')
dat.load(db)

In [9]:
def get_corpus(ds):
    """
    To flatten all abstract words into a long list, and make a corpus out of it 
    
    TO DO: Generalize to making a corpus of authors, or any metric the author wants
    
    Attributes
    ----------
    ds : Data structure returned by lisc when a query is submitted
    """
    aPreCorpus = []
    
    numArticles = ds.n_articles
    print('Iterating over %d articles for extracting words') %(ds.n_articles)
    for AIdx in xrange(numArticles):
        if ds.words[AIdx] != None:
            aPreCorpus.append(ds.words[AIdx])
        
    
    PreText = list(itertools.chain.from_iterable(aPreCorpus))
    
    Corpus = nltk.Text(PreText)
    
    return Corpus
        

def get_common_words(Corpus,NumWords):
    """
    Return the most common words in the concatenated article text corpus
    
    Attributes
    ----------
    Corpus: generated from keyword arguments from cognitive atlas
    NumWords: Top N words to return
    """
    
    assert(NumWords <= len(Corpus))
    
    fdist = nltk.FreqDist(Corpus)
    common_words = fdist.most_common(NumWords)
    header = [('Term', 'Frequency'),('-'*len('Term'),'-'*len('Frequency'))]
    wordList = header + common_words
    
    width = max(len(e) for t in wordList for e in t[:-1]) + 1 
    format=('%%-%ds' % width) * len(wordList[0])
    print '\n'.join(format % tuple(t) for t in wordList)
    print('\n Total Word Count : %d') %(len(Corpus))
    

def get_similarity(Corpus,ds,keyword='memory'):
    """
    Returns the most similar words, determined by neighborhood in corpus
    
    Attributes
    ----------
    
    Corpus : concatenated list of abstracts
    ds : ouptut of lisc
    keyword : query
    
    NOTE: There is a bug in text.similarity in NLTK2.0, we want to use this function to return similar 
    words in a ranked fashion 
    """
    #similar_words = Corpus.similar(keyword)
    #return similar_words
    #similar_words = Corpus._word_contex_index.similar_words(keyword)
    #if similar_words != None:
     #   print '\n'.join(word for word in similar_words)
    #else: 
     #   print 'No matches.'
        
    idx = nltk.text.ContextIndex([word.lower() for word in Corpus])
    simWords = []
    for word in nltk.word_tokenize(keyword):
        simWords.append(idx.similar_words(word))
        if word != None:
            print '\n'.join(idx.similar_words(word))
    return list(itertools.chain.from_iterable(simWords))

In [16]:
# See how many terms from Cognitive Atlas exist in given Corpus, don't run this now
CA_words = ['abstract reasoning','attention','arousal']
def CA_word_presence(Corpus,CA_terms):
    """
    Returns the words from CA that are present in the CA
    
    TO DO: Generalize to entering tasks 
    
    Attributes
    ----------
    
    Corpus : concatenated list of abstracts
    """    
    mask = []
    for ca_idx in xrange(len(CA_terms)):
        if CA_terms[ca_idx] in Corpus:
            mask.append(True)
        else:
            mask.append(False)
            
    PresentTerms = list(compress(CA_words, mask))
    print '\n'.join(word for word in PresentTerms)
    
    return PresentTerms
   

### Scratchpad

In [ ]:
Corpus = get_corpus(dat)

In [ ]:
get_common_words(Corpus,10)

In [ ]:
sim = get_similarity(Corpus,dat,'autobiographical recall')

In [ ]:
def test_neighbors(Corpus,keyword)
    
    plain_list = list(Corpus)
    max_instances = 10

    indices = [i for i, x in enumerate(plain_list) if x == "p300"]

    check_first = indices[:max_instances]

    for i in range(max_instances):
        print('Occurence %d of p300 preceded by %s and followed by %s.') \
        %(check_first[i],plain_list[i-1],plain_list[i+1])

### Visualization with Gensim

In [27]:
import gensim
from gensim import corpora
import pandas as pd
from gensim.models import LdaModel
import pyLDAvis.gensim

In [28]:
textList = []
    
numArticles = dat.n_articles
for AIdx in xrange(numArticles):
    if dat.words[AIdx] != None:
        textList.append(dat.words[AIdx])
        

In [29]:
dictionary = corpora.Dictionary(textList)
dictionary.save('dictionary.dict')

In [30]:
doc_term_matrix = [dictionary.doc2bow(doc) for doc in textList]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

In [31]:
Lda = gensim.models.ldamodel.LdaModel

ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

ldamodel.save('topic.model')

In [32]:
loading = LdaModel.load('topic.model')
print(loading.print_topics(num_topics=2, num_words=4))

[(0, u'0.026*"autobiographical" + 0.021*"memory" + 0.015*"recall" + 0.010*"memories"'), (1, u'0.018*"mood" + 0.012*"induction" + 0.009*"study" + 0.009*"j"')]


In [33]:
pyLDAvis.enable_notebook()

In [34]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')

In [35]:
data = pyLDAvis.gensim.prepare(lda, c, d)

In [36]:
data

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      42.782743        1       1 -0.079185  0.070177
2      36.953805        1       2 -0.045511 -0.084061
1      20.263452        1       3  0.124696  0.013884, topic_info=     Category        Freq              Term       Total  loglift  logprob
term                                                                     
1050  Default   52.000000              mood   52.000000  30.0000  30.0000
1911  Default   28.000000         induction   28.000000  29.0000  29.0000
2230  Default   15.000000                 j   15.000000  28.0000  28.0000
1789  Default   14.000000           psychol   14.000000  27.0000  27.0000
1140  Default   35.000000          semantic   35.000000  26.0000  26.0000
1755  Default   25.000000        depression   25.000000  25.0000  25.0000
839   Default   19.000000        rumination   19.000000  24.0000  24.0000
430   Default   26.000000          episodic   26.000000  23.0000  23.0000
2018  Default   12.000000        procedures   12.000000  22.0000  22.0000
81    Default   27.000000          negative   27.000000  21.0000  21.0000
1276  Default   38.000000         retrieval   38.000000  20.0000  20.0000
60    Default   43.000000             study   43.000000  19.0000  19.0000
1291  Default   13.000000          symptoms   13.000000  18.0000  18.0000
788   Default   27.000000           emotion   27.000000  17.0000  17.0000
299   Default    7.000000                de    7.000000  16.0000  16.0000
198   Default    7.000000          surprise    7.000000  15.0000  15.0000
513   Default    7.000000               soc    7.000000  14.0000  14.0000
1149  Default    6.000000          children    6.000000  13.0000  13.0000
246   Default    9.000000          behavior    9.000000  12.0000  12.0000
396   Default   13.000000         procedure   13.000000  11.0000  11.0000
1867  Default   27.000000          temporal   27.000000  10.0000  10.0000
76    Default    6.000000             moral    6.000000   9.0000   9.0000
1301  Default    6.000000       anticipated    6.000000   8.0000   8.0000
1842  Default    6.000000             moods    6.000000   7.0000   7.0000
2116  Default   21.000000            effect   21.000000   6.0000   6.0000
664   Default  127.000000            memory  127.000000   5.0000   5.0000
82    Default   16.000000             music   16.000000   4.0000   4.0000
2139  Default    8.000000      recollection    8.000000   3.0000   3.0000
1220  Default   22.000000               two   22.000000   2.0000   2.0000
1962  Default    4.000000              clin    4.000000   1.0000   1.0000
...       ...         ...               ...         ...      ...      ...
643    Topic3    2.981156             behav    3.522301   1.4295  -6.3251
172    Topic3    2.981155              ther    3.522301   1.4295  -6.3251
880    Topic3    2.981145           outrage    3.522303   1.4295  -6.3251
399    Topic3    2.981145         punishers    3.522303   1.4295  -6.3251
316    Topic3    2.981139              fair    3.522304   1.4295  -6.3251
1119   Topic3    2.981134              film    3.522305   1.4295  -6.3251
2255   Topic3    2.979672         traumatic    3.522544   1.4290  -6.3256
1911   Topic3   19.738031         induction   28.127569   1.2421  -4.4348
2018   Topic3    9.116598        procedures   12.146953   1.3094  -5.2073
246    Topic3    7.123157          behavior    9.270849   1.3328  -5.4540
1050   Topic3   30.032770              mood   52.142203   1.0447  -4.0151
652    Topic3    3.465842          inducing    4.249871   1.3924  -6.1744
327    Topic3    4.361987        techniques    5.698030   1.3292  -5.9444
1336   Topic3    4.359922             story    5.698368   1.3286  -5.9449
1291   Topic3    7.997517          symptoms   13.109188   1.1022  -5.3382
396    Topic3    8.170838         procedure   13.984425   1.0590  -5.3168
81     Topic3   12.106331          negative   27.780326   0.7